In [ ]:
# syft absolute
import syft as sy
from syft.service.code.user_code import UserCode
from syft.service.request.request import Request

In [ ]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
)

server_high = sy.orchestra.launch(
    name="bigquery-high",
    server_side_type="high",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
)

In [ ]:
low_client = server_low.login(email="info@openmined.org", password="changethis")
high_client = server_high.login(email="info@openmined.org", password="changethis")

# Sync, approve, sync

In [ ]:
# todo: this is way too noisy
widget = sy.sync(from_client=low_client, to_client=high_client)

In [ ]:
# this is not great
widget.click_sync(0)

In [ ]:
assert len(high_client.code.get_all()) == 1

In [ ]:
request = high_client.requests[0]

In [ ]:
def execute_request(client, request) -> dict:
    if not isinstance(request, Request):
        return "This is not a request"

    code = request.code
    if not isinstance(code, UserCode):
        return "No usercode found"

    func_name = request.code.service_func_name
    api_func = getattr(client.code, func_name, None)
    if api_func is None:
        return "Code name was not found on the client."

    job = api_func(blocking=False)
    return job

In [ ]:
job = execute_request(high_client, request)

In [ ]:
job

In [ ]:
job.wait()

In [ ]:
# third party
from sync_helpers import sync_finished_jobs

In [ ]:
sync_finished_jobs(client_low=low_client, client_high=high_client)

In [ ]:
low_client.requests

In [ ]:
server_high.land()
server_low.land()